In [35]:
import requests #pra capturar a info
from bs4 import BeautifulSoup #pra colocar no formato xml
import sys #pra usar num except
import traceback #pra trzer uma formataçao de erro diferenciada
import urllib
import spacy #esses dois abaixo pra poder pre-processar o texto
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.probability import FreqDist
import re #é o regex
from wordcloud import WordCloud
from matplotlib import pyplot as plt
from collections import Counter

In [36]:
def get_text (url): 
    buf = []

    try:
        soup = BeautifulSoup(requests.get(url).text, "html.parser") #vai pegar o text no site e deixar no formato 
                                                        #parecido com xml, onde vou usar um parser do tipo html. 

        for p in soup.find_all("p"): #daí, dentro do soup, eu vou fazer uma busca por tags "p" e add 
            buf.append(p.get_text()) # esse texto no buf

        return "\n".join(buf) #fazer um join pra pegar cada elemento dessa lista e transformar num só string
    
    except:
        print(traceback.format_exc()) #se nao der certo, ele sai do código e dá uma formatação específica
        sys.exit(-1)

In [60]:
text = get_text("https://edition.cnn.com/europe/live-news/russia-ukraine-war-news-09-05-22/index.html")
text = text.lower()
print(text)

live
russia's war in ukraine
live
americans kidnapped in mexico
live
louisville police report
live
what's moving markets today
by aditi sangal, amy woodyatt, simone mccarthy, tara subramaniam and maureen chowdhury, cnn
our live coverage has ended. read more about today’s developments in the posts below.
from cnn's yulia kesaieva
ukrainian president volodymyr zelensky claimed his country’s military has destroyed an important russian ammunition depot where moscow kept s-300 missiles it fired on the city of kharkiv.
 “good thing mlrs (multiple launch rocket systems). thank you, our heroes,” he concluded.
the ukrainian president went on to promise retribution for the russian shelling of several ukrainian cities and towns.
  
from cnn's yulia kesaieva
ukrainian president volodymyr zelenksy said he looks forward to working with incoming british prime minister liz truss, while at the same time thanking the uk’s outgoing leader, her predecessor, boris johnson, during his nightly address. 
“and

In [38]:
# !python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
x = "Hello, World"
y = nlp(x)
print(x,"\n",y)

Hello, World 
 Hello, World


In [39]:
print(type(x), "\n", type(y))

#reparar que o spacy transforma a string em tokens. Entao n se está mais trabalhando com str.

<class 'str'> 
 <class 'spacy.tokens.doc.Doc'>


In [40]:
def preprocess_spacy(string):
    #Deixar apenas elementos alfanuméricos
    string = re.sub(r"[^a-zA-Z0-9]+", " ", string)

    #tokenizar
    words = nlp(string)

    #remover stopwords. Lembrando que estamos no spacy. A list comprehension no NLTK seria um pouco diferente.
    #Vira meio que um atributo (.is_stop, .lemma_, etc.)
    filtered_words = [word for word in words if not word.is_stop]

    #aplicar a lematização (que diferente da stematização, vai reduzir as variações de uma palavra
    # a uma palavra existente e neutra. A stematização reduz a palavra a um radical invariável)
    lemma_words = [word.lemma_ for word in filtered_words]

    return lemma_words



In [41]:
#Tentando fazer o mesmo, só que com o NLTK:
from nltk.stem import WordNetLemmatizer
# nltk.download('omw-1.4')
sw = set(stopwords.words("english")) #estou criando uma lista com as stopwords em ingles. o set() me garante
#que essa lista tenha palavras únicas, pois pode ser que quando eu puxe a lista das stopwords, tenha palavras
#repetidas, afinal é alguem incluindo stopwords nela toda hora no mundo.
lemma = WordNetLemmatizer() #estou instanciando a classe de lematização

In [42]:
def preprocess_nltk(string):
    #Deixar apenas elementos alfanuméricos
    string = re.sub(r"[^a-zA-Z0-9]+", " ", string)

    #palavras em minusculo
    string = string.lower()

    #tokenizar
    words = word_tokenize(string)

    #remover stopwords.
    filtered_words = [word for word in words if word not in sw]

    #aplicar a lematização
    lemma_words = [lemma.lemmatize(word) for word in filtered_words]


    return lemma_words

In [55]:
text_spacy = preprocess_spacy(text)
text_nltk = preprocess_nltk(text) 

In [56]:
print(text_spacy)

['live', 'russia', 's', 'war', 'ukraine', 'live', 'americans', 'kidnap', 'mexico', 'live', 'louisville', 'police', 'report', 'live', 's', 'move', 'market', 'today', 'aditi', 'sangal', 'amy', 'woodyatt', 'simone', 'mccarthy', 'tara', 'subramaniam', 'maureen', 'chowdhury', 'cnn', 'live', 'coverage', 'end', 'read', 'today', 's', 'development', 'post', 'cnn', 's', 'yulia', 'kesaieva', 'ukrainian', 'president', 'volodymyr', 'zelensky', 'claim', 'country', 's', 'military', 'destroy', 'important', 'russian', 'ammunition', 'depot', 'moscow', 'keep', 's', '300', 'missile', 'fire', 'city', 'kharkiv', 'good', 'thing', 'mlrs', 'multiple', 'launch', 'rocket', 'system', 'thank', 'hero', 'conclude', 'ukrainian', 'president', 'go', 'promise', 'retribution', 'russian', 'shelling', 'ukrainian', 'city', 'town', 'cnn', 's', 'yulia', 'kesaieva', 'ukrainian', 'president', 'volodymyr', 'zelenksy', 'say', 'look', 'forward', 'work', 'incoming', 'british', 'prime', 'minister', 'liz', 'trus', 'time', 'thank', 'u

In [57]:
print(text_nltk)

['live', 'russia', 'war', 'ukraine', 'live', 'american', 'kidnapped', 'mexico', 'live', 'louisville', 'police', 'report', 'live', 'moving', 'market', 'today', 'aditi', 'sangal', 'amy', 'woodyatt', 'simone', 'mccarthy', 'tara', 'subramaniam', 'maureen', 'chowdhury', 'cnn', 'live', 'coverage', 'ended', 'read', 'today', 'development', 'post', 'cnn', 'yulia', 'kesaieva', 'ukrainian', 'president', 'volodymyr', 'zelensky', 'claimed', 'country', 'military', 'destroyed', 'important', 'russian', 'ammunition', 'depot', 'moscow', 'kept', '300', 'missile', 'fired', 'city', 'kharkiv', 'good', 'thing', 'mlrs', 'multiple', 'launch', 'rocket', 'system', 'thank', 'hero', 'concluded', 'ukrainian', 'president', 'went', 'promise', 'retribution', 'russian', 'shelling', 'several', 'ukrainian', 'city', 'town', 'cnn', 'yulia', 'kesaieva', 'ukrainian', 'president', 'volodymyr', 'zelenksy', 'said', 'look', 'forward', 'working', 'incoming', 'british', 'prime', 'minister', 'liz', 'truss', 'time', 'thanking', 'uk'

In [58]:
text_spacy_2 = " ".join(text_spacy)
text_spacy_2

'live russia s war ukraine live americans kidnap mexico live louisville police report live s move market today aditi sangal amy woodyatt simone mccarthy tara subramaniam maureen chowdhury cnn live coverage end read today s development post cnn s yulia kesaieva ukrainian president volodymyr zelensky claim country s military destroy important russian ammunition depot moscow keep s 300 missile fire city kharkiv good thing mlrs multiple launch rocket system thank hero conclude ukrainian president go promise retribution russian shelling ukrainian city town cnn s yulia kesaieva ukrainian president volodymyr zelenksy say look forward work incoming british prime minister liz trus time thank uk s outgoing leader predecessor boris johnson nightly address believe able lot protect nation thwart russian destructive effort add zelensky s warm welcome truss come bid farewell outgoing prime minister boris johnson speak early monday usually say friendship politic boris case understand half word friends

In [59]:
text_nltk_2 = " ".join(text_nltk)
text_nltk_2

'live russia war ukraine live american kidnapped mexico live louisville police report live moving market today aditi sangal amy woodyatt simone mccarthy tara subramaniam maureen chowdhury cnn live coverage ended read today development post cnn yulia kesaieva ukrainian president volodymyr zelensky claimed country military destroyed important russian ammunition depot moscow kept 300 missile fired city kharkiv good thing mlrs multiple launch rocket system thank hero concluded ukrainian president went promise retribution russian shelling several ukrainian city town cnn yulia kesaieva ukrainian president volodymyr zelenksy said look forward working incoming british prime minister liz truss time thanking uk outgoing leader predecessor boris johnson nightly address believe together able lot protect nation thwart russian destructive effort added zelensky warm welcome truss came bidding farewell outgoing prime minister boris johnson spoken earlier monday usually said friendship politics boris c